In [4]:
import pyspark
from pyspark import SparkContext
sc =SparkContext()

In [5]:
import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns

import re
import string

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer


In [6]:

#from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle
from sklearn.naive_bayes import MultinomialNB

#from xgboost import XGBClassifier

#from transformers import pipeline


In [7]:
#import all the libraries of pyspark.sql
from pyspark.sql import*
#import SparkContext and SparkConf
from pyspark import SparkContext, SparkConf

In [8]:
sqlContext = SQLContext(sc)

In [10]:
df=sqlContext.read.csv("AmazonReviewData.csv", header=True)

In [120]:
df.count()

34658

In [11]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- asins: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- keys: string (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- ReviewsDate: string (nullable = true)
 |-- ReviewsdateAdded: string (nullable = true)
 |-- ReviewsDateSeen: string (nullable = true)
 |-- reviews.didPurchase: string (nullable = true)
 |-- reviews.doRecommend: string (nullable = true)
 |-- reviews.id: string (nullable = true)
 |-- reviews.numHelpful: string (nullable = true)
 |-- reviews.rating: string (nullable = true)
 |-- reviews.text: string (nullable = true)
 |-- reviews.title: string (nullable = true)
 |-- reviews.userCity: string (nullable = true)
 |-- reviews.userProvince: string (nullable = true)
 |-- reviews.username: string (nullable = true)



In [12]:
df = df.withColumnRenamed("reviews.text","ReviewText")

In [13]:
df = df.withColumnRenamed("reviews.rating","ReviewRating")

In [14]:
df1 = df.withColumnRenamed("ReviewRating","Rating")
df1 = df.withColumnRenamed("ReviewText","ReviewText")

In [125]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- asins: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- keys: string (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- ReviewsDate: string (nullable = true)
 |-- ReviewsdateAdded: string (nullable = true)
 |-- ReviewsDateSeen: string (nullable = true)
 |-- reviews.didPurchase: string (nullable = true)
 |-- reviews.doRecommend: string (nullable = true)
 |-- reviews.id: string (nullable = true)
 |-- reviews.numHelpful: string (nullable = true)
 |-- ReviewRating: string (nullable = true)
 |-- ReviewText: string (nullable = true)
 |-- reviews.title: string (nullable = true)
 |-- reviews.userCity: string (nullable = true)
 |-- reviews.userProvince: string (nullable = true)
 |-- reviews.username: string (nullable = true)



In [15]:
df1 = df1.drop("id","Name","asins","brand","categories","keys","manufacturer","reviews.date","reviews.dateAdded","reviews.dateSeen")

In [24]:
df1.printSchema()

root
 |-- reviews.didPurchase: string (nullable = true)
 |-- reviews.doRecommend: string (nullable = true)
 |-- reviews.id: string (nullable = true)
 |-- reviews.numHelpful: string (nullable = true)
 |-- ReviewRating: string (nullable = true)
 |-- reviews.sourceURLs: string (nullable = true)
 |-- ReviewText: string (nullable = true)
 |-- reviews.title: string (nullable = true)
 |-- reviews.userCity: string (nullable = true)
 |-- reviews.userProvince: string (nullable = true)
 |-- reviews.username: string (nullable = true)



In [16]:
df1 = df1.drop("reviews.didPurchase","reviews.doRecommend","reviews.id","reviews.numHelpful","reviews.sourceURLs")

In [17]:
df1 = df1.drop("reviews.title","reviews.userCity","reviews.userProvince","reviews.username")

In [18]:
df1 = df1.drop("reviewsDate","ReviewsdateAdded","ReviewsDateSeen","reviews.username")

In [19]:
df1.printSchema()

root
 |-- ReviewRating: string (nullable = true)
 |-- ReviewText: string (nullable = true)



In [20]:
df1.show(5)

+------------+--------------------+
|ReviewRating|          ReviewText|
+------------+--------------------+
|           5|This product so f...|
|           5|great for beginne...|
|           5|Inexpensive table...|
|           4|I ve had my Fire ...|
|           5|I bought this for...|
+------------+--------------------+
only showing top 5 rows



In [134]:
df1.count()

34658

In [31]:
df1.distinct().count()

27941

In [23]:
from pyspark.sql.functions import col,isnan,when,count
df1.select([count(when(isnan(c), c)).alias(c) for c in df1.columns]).show()

+------------+----------+
|ReviewRating|ReviewText|
+------------+----------+
|           0|         0|
+------------+----------+



In [21]:
df1.registerTempTable('table')

In [151]:
sqlContext.sql('select ReviewRating,count(*) from table group by ReviewRating order by ReviewRating desc').show(10)

+------------+--------+
|ReviewRating|count(1)|
+------------+--------+
|           5|   23775|
|           4|    8540|
|           3|    1498|
|           2|     402|
|           1|     410|
|           0|      33|
+------------+--------+



In [24]:
df1 = df1.withColumn("SentimentScore",when(df1.ReviewRating > 3,1).otherwise(0))

In [160]:
df1.show(10)

+------------+--------------------+--------------+
|ReviewRating|          ReviewText|SentimentScore|
+------------+--------------------+--------------+
|           5|This product so f...|             1|
|           5|great for beginne...|             1|
|           5|Inexpensive table...|             1|
|           4|I ve had my Fire ...|             1|
|           5|I bought this for...|             1|
|           5|This amazon fire ...|             1|
|           4|Great for e-readi...|             1|
|           5|I gave this as a ...|             1|
|           5|Great as a device...|             1|
|           5|I love ordering b...|             1|
+------------+--------------------+--------------+
only showing top 10 rows



In [25]:
df1 = df1.withColumn("Sentiment",when(df1.SentimentScore == 1,"Positive").otherwise("Negative"))

df1.show(10)

In [26]:
df1.show(10)

+------------+--------------------+--------------+---------+
|ReviewRating|          ReviewText|SentimentScore|Sentiment|
+------------+--------------------+--------------+---------+
|           5|This product so f...|             1| Positive|
|           5|great for beginne...|             1| Positive|
|           5|Inexpensive table...|             1| Positive|
|           4|I ve had my Fire ...|             1| Positive|
|           5|I bought this for...|             1| Positive|
|           5|This amazon fire ...|             1| Positive|
|           4|Great for e-readi...|             1| Positive|
|           5|I gave this as a ...|             1| Positive|
|           5|Great as a device...|             1| Positive|
|           5|I love ordering b...|             1| Positive|
+------------+--------------------+--------------+---------+
only showing top 10 rows



In [169]:
df1 = df1.repartition(1)
df1.write.csv('UpdatedWithSentiment.csv',header = 'true')

In [27]:
def clean_text(text:str):
  
    text = str(text)
    
    text = text.lower()
    text = text.strip()
    
    text = re.sub(' \d+', ' ', text)
    text = re.compile('<.*?>').sub('', text)
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)
    text = re.sub('\s+', ' ', text)
    
    text = text.strip()
    
    return text

In [172]:
text = "   This is a message to be cleaned. It may involve some things like: <br>, ?, :, ''  26 adjacent spaces and tabs     .  "
print(text, '\n')
clean_text(text)

   This is a message to be cleaned. It may involve some things like: <br>, ?, :, ''  26 adjacent spaces and tabs     .   



'this is a message to be cleaned it may involve some things like adjacent spaces and tabs'

In [28]:
def remove_stopwords(text:str):
  
    text = str(text)
    filtered_sentence = []

    stop_words = ["a", "an", "the", "this", "that", "is", "it", "to", "and"]

    words = word_tokenize(text)
    for w in words:
        if w not in stop_words:
            filtered_sentence.append(w)
    text = " ".join(filtered_sentence)
    
    return text

In [176]:
text = "   This is a message to be cleaned. It may involve some things like: <br>, ?, :, ''  26 adjacent spaces and tabs     .  "
print(text, '\n')
text = clean_text(text)
remove_stopwords(text)

   This is a message to be cleaned. It may involve some things like: <br>, ?, :, ''  26 adjacent spaces and tabs     .   



'message be cleaned may involve some things like adjacent spaces tabs'

In [175]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/vinu/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [29]:
def get_wordnet_pos(treebank_tag):
    
    if treebank_tag.startswith('J'):
            return 'a'
    elif treebank_tag.startswith('V'):
            return 'v'
    elif treebank_tag.startswith('N'):
            return 'n'
    elif treebank_tag.startswith('R'):
            return 'r'
    else:
    # As default pos in lemmatization is Noun
        return 'n'

In [178]:
def lemmatize(text:str):
    """ lemmatize text:
    ------
    input: text (str)    
    output: lemmatized text (str)
    """
    text = str(text)
    
    # Initialize the lemmatizer
    wl = WordNetLemmatizer()

    lemmatized_sentence = []

    # Tokenize the sentence
    words = word_tokenize(text)
    # Get position tags
    word_pos_tags = nltk.pos_tag(words)
    # Map the position tag and lemmatize the word/token
    for idx, tag in enumerate(word_pos_tags):
        lemmatized_sentence.append(wl.lemmatize(tag[0], get_wordnet_pos(tag[1])))

    lemmatized_text = " ".join(lemmatized_sentence)
    return lemmatized_text

In [30]:
def lemmatize1(data_str):
    # expects a string
    list_pos = 0
    cleaned_str = ''
    lmtzr = WordNetLemmatizer()
    #text = data_str.split()
    tagged_words = nltk.pos_tag(data_str)
    for word in tagged_words:
        lemma = lmtzr.lemmatize(word[0], get_wordnet_pos(word[1]))
        if list_pos == 0:
            cleaned_str = lemma
        else:
            cleaned_str = cleaned_str + ' ' + lemma
        list_pos += 1
    return cleaned_str

In [183]:
text = "   This is a message to be cleaned. It may involve some things like: <br>, ?, :, ''  26 adjacent spaces and tabs     .  "
print(text, '\n')
text = clean_text(text)
text = remove_stopwords(text)
# text = stemm_text(text)
lemmatize(text)

   This is a message to be cleaned. It may involve some things like: <br>, ?, :, ''  26 adjacent spaces and tabs     .   



'message be clean may involve some thing like adjacent space tabs'

In [180]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/vinu/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [182]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/vinu/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [31]:
from pyspark.sql.functions import udf

In [32]:
from pyspark.sql.types import IntegerType,StringType

In [33]:
CleanTextUDF = udf(lambda z:clean_text(z),StringType())

In [41]:
df3 = df1.withColumn('ReviewText',CleanTextUDF('ReviewText'))

In [35]:
RemoveStopwordsUDF = udf(lambda z:remove_stopwords(z),StringType())

In [43]:
df3 = df3.withColumn('ReviewText',RemoveStopwordsUDF('ReviewText'))

In [37]:
lemmatizeUDF1 = udf(lambda z:lemmatize1(z),StringType())

In [39]:
df3 = df3.withColumn('ReviewText',lemmatizeUDF1('ReviewText'))

In [191]:
nltk.download('omw')

[nltk_data] Downloading package omw to /Users/vinu/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.


True

In [44]:
df3.show(10)

+------------+--------------------+--------------+---------+
|ReviewRating|          ReviewText|SentimentScore|Sentiment|
+------------+--------------------+--------------+---------+
|           5|product so far ha...|             1| Positive|
|           5|great for beginne...|             1| Positive|
|           5|inexpensive table...|             1| Positive|
|           4|i ve had my fire ...|             1| Positive|
|           5|i bought for my g...|             1| Positive|
|           5|amazon fire inch ...|             1| Positive|
|           4|great for e readi...|             1| Positive|
|           5|i gave as christm...|             1| Positive|
|           5|great as device r...|             1| Positive|
|           5|i love ordering b...|             1| Positive|
+------------+--------------------+--------------+---------+
only showing top 10 rows



In [202]:
df2 = df2.repartition(1)
df2.write.csv('CleanedData.csv',header = 'true')

In [203]:
(train_set, val_set, test_set) = df3.randomSplit([0.98, 0.01, 0.01], seed = 2000)

In [204]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

In [205]:
tokenizer = Tokenizer(inputCol="ReviewText", outputCol="words")

In [206]:
hashtf = HashingTF(numFeatures=2**16, inputCol="words", outputCol='tf')

In [207]:
idf = IDF(inputCol='tf', outputCol="features", minDocFreq=5)

In [208]:
label_stringIdx = StringIndexer(inputCol = "SentimentScore", outputCol = "label")

In [209]:
pipeline = Pipeline(stages=[tokenizer, hashtf, idf, label_stringIdx])

In [210]:
pipelineFit = pipeline.fit(train_set)
train_df = pipelineFit.transform(train_set)
val_df = pipelineFit.transform(val_set)
train_df.show(5)

+------------+--------------------+--------------+---------+--------------------+--------------------+--------------------+-----+
|ReviewRating|          ReviewText|SentimentScore|Sentiment|               words|                  tf|            features|label|
+------------+--------------------+--------------+---------+--------------------+--------------------+--------------------+-----+
|           0|all of them quit ...|             0| Negative|[all, of, them, q...|(65536,[3085,5914...|(65536,[3085,5914...|  1.0|
|           0|an amazon com off...|             0| Negative|[an, amazon, com,...|(65536,[1880,3085...|(65536,[1880,3085...|  1.0|
|           0|as reviewed by th...|             0| Negative|[as, reviewed, by...|(65536,[619,637,8...|(65536,[619,637,8...|  1.0|
|           0|bought for my hus...|             0| Negative|[bought, for, my,...|(65536,[1714,3411...|(65536,[1714,3411...|  1.0|
|           0|       great love it|             0| Negative|   [great, love, it]|(65536,[3

In [211]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(maxIter=100)
lrModel = lr.fit(train_df)
predictions = lrModel.transform(val_df)

In [212]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator.evaluate(predictions)

0.6646445037612239

In [213]:
accuracy = predictions.filter(predictions.label == predictions.prediction).count() / float(val_set.count())
accuracy

0.9067055393586005